In [8]:
from pyspark.sql import SparkSession
from typing import Generator, List
import pandas as pd 
import os 

spark = SparkSession.builder.master("local").appName("year-taxi").getOrCreate()

# directory: Generator[str, None, None] = (f"{os.getcwd()}/data/{i}" for i in range(1, len(os.listdir(f"{os.getcwd()}/data"))))
# filename: List[str] = [f"{p}/{data}" for p in directory for data in os.listdir(p)]

In [9]:
directory: str = f"{os.getcwd()}/data/2020"
trip3: str = "fhvhv_tripdata_2020-03.parquet"

trip_data3 = spark.read.parquet(f"file:///{directory}/{trip3}")
trip_data3.createOrReplaceTempView('month3_data')

In [15]:
import matplotlib.pyplot as plt 
# pickup_datetime pickup_datetime

qs = """
select 
    pickup, count(*) as trips
from 
    (select split(pickup_datetime, " ")[0] as pickup from month3_data)
group by 
    pickup
order by 
    pickup asc
"""
spark.sql(qs).show()

+----------+------+
|    pickup| trips|
+----------+------+
|2020-03-01|445657|
|2020-03-02|677283|
|2020-03-03|642542|
|2020-03-04|710968|
|2020-03-05|706957|
|2020-03-06|783893|
|2020-03-07|894951|
|2020-03-08|875015|
|2020-03-09|651566|
|2020-03-10|619615|
|2020-03-11|625082|
|2020-03-12|633048|
|2020-03-13|658978|
|2020-03-14|605978|
|2020-03-15|532083|
|2020-03-16|418197|
|2020-03-17|350059|
|2020-03-18|285114|
|2020-03-19|264873|
|2020-03-20|243352|
+----------+------+
only showing top 20 rows

